In [25]:
%matplotlib inline
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms, datasets, models
import warnings
warnings.filterwarnings('ignore')

In [26]:
np.random.seed(0)

In [27]:
!ls ../../Downloads/pretrained-pytorch-models

densenet161-17b70270.pth	  resnet50-19c8e357.pth
inception_v3_google-1a9a5a14.pth  squeezenet1_0-a815701f.pth
pretrained-pytorch-models.zip	  squeezenet1_1-f364aa15.pth
resnet18-5c106cde.pth


In [28]:
cache_dir = expanduser(join('~', '.torch'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [29]:
!cp ../../Downloads/pretrained-pytorch-models/* /home/chenxi-zhang/.torch/models/

In [30]:
!ls /home/chenxi-zhang/.torch/models/

densenet161-17b70270.pth	  resnet50-19c8e357.pth
inception_v3_google-1a9a5a14.pth  squeezenet1_0-a815701f.pth
pretrained-pytorch-models.zip	  squeezenet1_1-f364aa15.pth
resnet18-5c106cde.pth


In [31]:
!ls ../../Downloads/Dogs_Breed/

labels.csv  sample_submission.csv  test  test.zip  train  train.zip


Using just 16 most frequent breeds to keep the running time under the kernel limit

In [32]:
INPUT_SIZE = 224
NUM_CLASSES = 16
data_dir = '../../Downloads/Dogs_Breed/'
labels = pd.read_csv(join(data_dir, 'labels.csv'))
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


In [33]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)

train = labels_pivot.sample(frac=0.8)
valid = labels_pivot[~labels_pivot['id'].isin(train['id'])]
print(train.shape, valid.shape)

TypeError: 'NoneType' object is not callable